In [1]:
corpus = sc.textFile("WikipediaPagesOneDocPerLine1000LinesSmall.txt")

In [2]:
numberOfDocs = corpus.count()

In [3]:
numberOfDocs

1000

In [4]:
# Each entry in validLines will be a line from the text file
validLines = corpus.filter(lambda x : 'id' in x and 'url=' in x)

In [5]:
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))

In [6]:
keyAndText.take(5)

[('431949',
  'Black people and MormonismWhile at least two black men held the priesthood in the early church, from the mid-1800s until 1978, The Church of Jesus Christ of Latter-day Saints (LDS Church) had a policy which prevented most men of black African descent from being ordained to the church\'s lay priesthood. Under the temple and priesthood restrictions before 1978, black members of African descent could not receive the priesthood or participate in temple ordinances besides baptisms for the dead. For a time in the 1960s and 1970s, they were not allowed to perform baptisms for the dead either. For young men and men in the LDS church, holding the priesthood is required to hold leadership roles, perform baptisms, to bless the sacrament, to bless babies and to bless the sick. Since black men could not hold the priesthood, they were excluded from holding leadership roles and performing these rituals. Temple ordinances are necessary for members to receive the endowment and marriage s

In [7]:
import re
regex = re.compile('[^a-zA-Z]')

In [8]:
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

In [9]:
keyAndListOfWords.take(5)

[('431949',
  ['black',
   'people',
   'and',
   'mormonismwhile',
   'at',
   'least',
   'two',
   'black',
   'men',
   'held',
   'the',
   'priesthood',
   'in',
   'the',
   'early',
   'church',
   'from',
   'the',
   'mid',
   's',
   'until',
   'the',
   'church',
   'of',
   'jesus',
   'christ',
   'of',
   'latter',
   'day',
   'saints',
   'lds',
   'church',
   'had',
   'a',
   'policy',
   'which',
   'prevented',
   'most',
   'men',
   'of',
   'black',
   'african',
   'descent',
   'from',
   'being',
   'ordained',
   'to',
   'the',
   'church',
   's',
   'lay',
   'priesthood',
   'under',
   'the',
   'temple',
   'and',
   'priesthood',
   'restrictions',
   'before',
   'black',
   'members',
   'of',
   'african',
   'descent',
   'could',
   'not',
   'receive',
   'the',
   'priesthood',
   'or',
   'participate',
   'in',
   'temple',
   'ordinances',
   'besides',
   'baptisms',
   'for',
   'the',
   'dead',
   'for',
   'a',
   'time',
   'in',
   

In [10]:
allWords =keyAndListOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))

In [11]:
allWords.take(5)

[('black', 1), ('people', 1), ('and', 1), ('mormonismwhile', 1), ('at', 1)]

In [12]:
from operator import add
allCounts = allWords.reduceByKey(add)

In [13]:
allCounts.take(5)

[('at', 4810), ('two', 1848), ('priesthood', 78), ('in', 27758), ('mid', 159)]

In [14]:
topWords = allCounts.top(1000, key=lambda x: x[1])

In [15]:
topWords

[('the', 74530),
 ('of', 34512),
 ('and', 28479),
 ('in', 27758),
 ('to', 22583),
 ('a', 21212),
 ('was', 12160),
 ('as', 8811),
 ('for', 8773),
 ('on', 8435),
 ('is', 8392),
 ('s', 8291),
 ('by', 7483),
 ('with', 7108),
 ('that', 6935),
 ('he', 5552),
 ('from', 5216),
 ('his', 5041),
 ('at', 4810),
 ('it', 4442),
 ('an', 3900),
 ('were', 3587),
 ('which', 3459),
 ('are', 3363),
 ('be', 3054),
 ('or', 2954),
 ('this', 2922),
 ('had', 2848),
 ('also', 2466),
 ('not', 2333),
 ('one', 2274),
 ('first', 2197),
 ('its', 2146),
 ('their', 2143),
 ('but', 2142),
 ('after', 2125),
 ('new', 2121),
 ('has', 2100),
 ('have', 2058),
 ('they', 1896),
 ('two', 1848),
 ('been', 1839),
 ('who', 1791),
 ('during', 1632),
 ('other', 1607),
 ('all', 1523),
 ('when', 1514),
 ('time', 1402),
 ('into', 1386),
 ('more', 1325),
 ('there', 1282),
 ('would', 1228),
 ('only', 1184),
 ('can', 1182),
 ('some', 1180),
 ('may', 1162),
 ('years', 1158),
 ('she', 1152),
 ('most', 1149),
 ('such', 1115),
 ('her', 1095)

In [16]:
twentyK = sc.parallelize(range(1000))

In [17]:
dictionary = twentyK.map (lambda x : (topWords[x][0], x))

In [18]:
dictionary.take(5)

[('the', 0), ('of', 1), ('and', 2), ('in', 3), ('to', 4)]

In [19]:
allWords = keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

In [20]:
allWords.take(5)

[('black', '431949'),
 ('people', '431949'),
 ('and', '431949'),
 ('mormonismwhile', '431949'),
 ('at', '431949')]

In [21]:
allDictionaryWords = dictionary.join(allWords)

In [22]:
allDictionaryWords.take(1)

[('of', (1, '431949'))]

In [23]:
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))

In [24]:
justDocAndPos.take(5)

[('431949', 1), ('431949', 1), ('431949', 1), ('431949', 1), ('431949', 1)]

In [25]:
import numpy as np

In [26]:
def stringVector (x):
    returnVal= str(x[0])
    for j in x[1]:
        returnVal += ','+ str(j)
    return returnVal

In [27]:
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

In [28]:
allDictionaryWordsInEachDoc.take(5)

[('431971', <pyspark.resultiterable.ResultIterable at 0x110cf3908>),
 ('431999', <pyspark.resultiterable.ResultIterable at 0x110cf3cc0>),
 ('432000', <pyspark.resultiterable.ResultIterable at 0x110cf3a58>),
 ('432001', <pyspark.resultiterable.ResultIterable at 0x110cf3f28>),
 ('432035', <pyspark.resultiterable.ResultIterable at 0x110cf3518>)]

In [29]:
forCSV= allDictionaryWordsInEachDoc.mapValues(lambda x: np.array( list(x)))

In [30]:
print(forCSV.take(5))

[('431971', array([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   1,   7,   7,   7,   7,   7,   7,   7,   7,   7,   7,
         7,  11,  11,  11,  36,  38,  40,  40,  40,  50,  68,  68,  95,
        95,  95, 117, 117, 117, 157, 162, 162, 162, 278, 278, 278, 393,
       518, 538, 554, 619, 904,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   4,
         4,   4,   4,   4,   4,  12,  12,  12,  12,  12,  12,  12,  12,
        12,  12,  12,  12,  12,  12,  12,  12,  12,  12,  12,  12,  12,
        12,  12,  12,  12,  12,  14,  25,  30,  30,  30,  31,  31,  32,
        37,  37,  41,  41,  75,  99, 154, 154, 167, 237, 256, 261, 261,
       292, 292, 353, 520, 752, 752, 752, 766,   3,   3,   3,   3,   3,
         3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,
         3,   3,   3,   6,   6,   6,   6,   6,   6, 

In [31]:
forCSV= forCSV.map(lambda x: stringVector(x))

In [32]:
print(forCSV.take(2))

['431971,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,7,7,7,7,7,7,7,7,7,7,11,11,11,36,38,40,40,40,50,68,68,95,95,95,117,117,117,157,162,162,162,278,278,278,393,518,538,554,619,904,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,4,4,4,4,4,4,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,12,14,25,30,30,30,31,31,32,37,37,41,41,75,99,154,154,167,237,256,261,261,292,292,353,520,752,752,752,766,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,18,43,49,56,65,65,100,100,100,336,348,348,354,419,646,719,719,817,834,951,960,960,960,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,8,8,8,8,13,13,13,13,13,13,13,16,19,19,19,19,19,28,39,39,45,62,64,64,77,77,88,88,133,164,205,205,205,233,233,233,233,233,233,233,233,233,254,339,381,432,432,458,514,514,514,533,546,656,702,711,746,746,746,757,757,757,983,10,17,17,17,17,35,35,93,97,110,110,110,110,110,110,145,145,177,177,192,

In [ ]:
#forCSV.saveAsTextFile(sys.argv[2]+"_allDictionaryWordsInEachDoc")